In [ ]:
import numpy as np
import pandas as pd
import itertools
import glob
import peptide_forest
pd.set_option("max_columns", 1000)
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:
import ursgal
ursgal.__version__

# Reading percolator outputs

In [ ]:
dfs = []
for csv in glob.glob("../data/E32/*percolator_3_4_0_validated.csv"):
    _df = pd.read_csv(csv)
#     _df['csv'] = csv
    for e in ['mascot', 'msfragger', 'msgfplus', 'omssa', 'xtandem']:
        if e in csv:
            _df['engine'] = e
    dfs.append(_df)
df = pd.concat(dfs)

In [ ]:
df.describe()

In [ ]:
df['Modifications'].fillna('', inplace=True)


In [ ]:
def create_seqmod(r):
    if r['Modifications'] == '':
        r_value = r['Sequence']
    else:
        r_value = r['Sequence'] + "#" + r['Modifications']
    return r_value

df['seq#mod'] = df.apply(create_seqmod, axis=1)
df.head()

In [ ]:
for name, grp in df.groupby('engine'):
    print(name, grp[(~grp['Is decoy']) & (grp['q-value'] < 0.01)]['seq#mod'].nunique())

In [ ]:
uc = ursgal.UController(verbose=False)
venn_main = uc.unodes['venndiagram_1_1_0']['class'].import_engine_as_python_function()
# ^-- this is ugly and our AirFlow nodes would make this prettier ! :)

In [ ]:
data = []
for name, grp in df.groupby('engine'):
    data.append(
        {
            'label': name,
            'data': set(grp[(~grp['Is decoy']) & (grp['q-value'] < 0.01)]['seq#mod'].tolist())
        }
    )

results = venn_main(
    data=data,
#     **default_kwargs
)


In [ ]:
from IPython.core.display import SVG
SVG(filename="VennDiagram.svg")

# reading peptide forest output

In [ ]:
output = "../01Apr_E13.csv"
output = "../01Apr_E32.csv"
final_df = pd.read_csv(output)
final_df.Modifications.fillna("", inplace=True)
final_df['seq#mod'] = final_df.apply(create_seqmod, axis=1)

In [ ]:
final_df.head()

In [ ]:
all_eng = [
    c.split("Score_processed_")[1] for c in final_df.columns if "Score_processed" in c
]

In [ ]:
# f2df = pd.read_csv("../25March_percolator_input.csv")
# f2df.Modifications.fillna("", inplace=True)
# f2df['seq#mod'] = f2df.apply(create_seqmod, axis=1)

In [ ]:
# f3df = pd.read_csv("../25March_no_msgfplus.csv")
# f3df.Modifications.fillna("", inplace=True)
# f3df['seq#mod'] = f3df.apply(create_seqmod, axis=1)

In [ ]:
q_val_cuts = np.logspace(-4, -1, num=10)
print(q_val_cuts)


In [ ]:
data = []
for eng in all_eng:
    y = []
    for q_value in q_val_cuts:
        y.append(
            final_df[(~final_df['Is decoy']) & (final_df[f'q-value_{eng}'] < q_value)]['seq#mod'].nunique()
        )
    data.append(
        go.Scatter(
            x=q_val_cuts, 
            y=y,
            mode='lines+markers',
            name="{0}".format(eng)
        )
    )

# for eng in all_eng:
#     y = []
#     for q_value in q_val_cuts:
#         y.append(
#             f2df[(~f2df['Is decoy']) & (f2df[f'q-value_{eng}'] < q_value)]['seq#mod'].nunique()
#         )
#     data.append(
#         go.Scatter(
#             x=q_val_cuts, 
#             y=y,
#             mode='lines+markers',
#             name="f2:{0}".format(eng)
#         )
#     )    

# for eng in all_eng:
#     y = []
#     if f'q-value_{eng}' not in f3df.columns:
#         continue
#     for q_value in q_val_cuts:
#         y.append(
#             f2df[(~f3df['Is decoy']) & (f3df[f'q-value_{eng}'] < q_value)]['seq#mod'].nunique()
#         )
#     data.append(
#         go.Scatter(
#             x=q_val_cuts, 
#             y=y,
#             mode='lines+markers',
#             name="{0}-training-without_msgf+".format(eng)
#         )
#     )        
    
for name, grp in df.groupby('engine'):
    y = []
    for q_value in q_val_cuts:
        y.append(
            grp[(~grp['Is decoy']) & (grp['q-value'] < q_value)]['seq#mod'].nunique()
        )
    data.append(
        go.Scatter(
            x=q_val_cuts, 
            y=y,
            mode='lines+markers',
            name="{0}:percolator v3.4".format(name)
        )
    ) 

In [ ]:
fig = go.Figure(data=data)
fig.update_layout(xaxis_type="log")
fig.show()

In [ ]:
data = []
for name in ['msgfplus', 'mascot']:
    grp = df[df['engine'] == name]
    data.append(
        {
            'label': "{0}:percolator v3.4".format(name),
            'data': set(grp[(~grp['Is decoy']) & (grp['q-value'] < 0.01)]['seq#mod'].tolist())
        }
    )

for name in ['RF-reg', 'mascot_1_0_0', 'msgfplus_v2018_06_28']:
    data.append(
        {
            'label': name,
            'data': set(final_df[(~final_df['Is decoy']) & (final_df[f'q-value_{name}'] < 0.01)]['seq#mod'].tolist())            
        }
)
    
results = venn_main(
    data=data,
#     **default_kwargs
)


In [ ]:
from IPython.core.display import SVG
SVG(filename="VennDiagram.svg")